In [ ]:
# Import local packages
import os
import sys
import glob

# Third party packages
import iris
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import iris.quickplot as qplt
import iris.plot as iplt
import datetime
import shutil
from six.moves import urllib
from pathlib import Path
import trackpy
from iris.time import PartialDateTime
import functions
import dask
import dask.array as da
import dask.distributed as dd
from dask import delayed
import cartopy.crs as ccrs
import xarray as xr
import netCDF4 as nc
import scipy
from scipy import ndimage
from scipy.ndimage import label, generate_binary_structure
import tobac #tobac package cloned from https://github.com/tobac-project/tobac.git


In [ ]:
# Import and set up warnings
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore', category=pd.io.pytables.PerformanceWarning)
warnings.filterwarnings('ignore')

In [ ]:
# Import the functions and dictionaries
import functions as fnc
import dictionaries as dic

In [ ]:
# Open the datasets
mask, precip, tracks = fnc.open_datasets(dic.mask_file, dic.precip_file, dic.tracks_file)

In [ ]:
# Create a copy of the tracks file
tracks = fnc.copy_tracks_file(tracks)

In [ ]:
# Add the precip columns to tracks
tracks = fnc.add_precip_columns(tracks)

In [ ]:
# Remove non track cells from tracks
tracks = fnc.remove_non_track_cells(tracks)

In [ ]:
# Find all of the unique track cells values within the tracks df
unique_cells = fnc.find_unique_cells(tracks)

In [ ]:
# Create a list of delayed computations for each cell
delayed_results = [delayed(fnc.precip_filtering_loop_cell)(cell, tracks, precip, mask, unique_cells, dic.precip_threshold, dic.heavy_precip_threshold, dic.extreme_precip_threshold, dic.s, dic.precip_area, dic.removed_tracks, dic.tracks_filtered_output, dic.tracks_cell_output_dir) for cell in unique_cells]

# compute the results in parallel
tracks_list = dask.compute(*delayed_results)

In [ ]:
# Concatenate the results back into a single dataframe
# using a dask dataframe
tracks = pd.concat(tracks_list)